In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

from datetime import datetime
import pytz

# spark = SparkSession.builder.appName('Lab').getOrCreate()
# data = [("Alice", 25), ("Bob", 30), ("Charlie", 28)]
# col = ["name", "age"]

# data = [(1, "foo"),(2, "bar")]
# col = ["id", "name"]
# df = spark.createDataFrame([(1, "foo"),(2, "bar")], ["id", "name"])
# df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)



In [17]:
path = '../data/transaction.csv'
df = spark.read.csv(path, sep='|', header=True, inferSchema=True)

In [20]:
df.schema

StructType([StructField('transactionId', StringType(), True), StructField('custId', IntegerType(), True), StructField('transactionDate', DateType(), True), StructField('productSold', StringType(), True), StructField('unitsSold', IntegerType(), True)])

In [ ]:
df.printSchema()

In [757]:
df.show(10)

+----------------+------+---------------+-----------+---------+
|   transactionId|custId|transactionDate|productSold|unitsSold|
+----------------+------+---------------+-----------+---------+
|20110111-0023586| 23586|     2011-01-11|    DETA800|      176|
|20110111-0023586| 23586|     2011-01-11|    SUPA101|      193|
|20110111-0023586| 23586|     2011-01-11|    SUPA102|      157|
|20110113-0023524| 23524|     2011-01-13|    PURA250|      122|
|20110113-0023844| 23844|     2011-01-13|    SUPA101|      217|
|20110114-0023968| 23968|     2011-01-14|    PURA100|      284|
|20110119-0023582| 23582|     2011-01-19|    PURA100|      184|
|20110119-0023582| 23582|     2011-01-19|    PURA250|      178|
|20110119-0023582| 23582|     2011-01-19|    SUPA101|      173|
|20110119-0023582| 23582|     2011-01-19|    SUPA101|      173|
+----------------+------+---------------+-----------+---------+
only showing top 10 rows



In [759]:
type(df)

pyspark.sql.dataframe.DataFrame

In [714]:
df.filter((f.col("productSold") == 'DETA800') & (f.col("custId") == '23262')).agg(f.sum('unitsSold')).show()

+--------------+
|sum(unitsSold)|
+--------------+
|           654|
+--------------+



# FavProduct

In [715]:
# df2 = df.groupBy('custId','productSold').sum('unitsSold').withColumnRenamed("sum(unitsSold)", "totalUnitsSold").orderBy('custId')
df2 = df.groupBy('custId','productSold').agg(F.sum('unitsSold').alias('totalUnitsSold')).orderBy('custId')

In [716]:
df2.show(10)

+------+-----------+--------------+
|custId|productSold|totalUnitsSold|
+------+-----------+--------------+
| 23262|    SUPA102|           204|
| 23262|    DETA200|           129|
| 23262|    PURA100|           492|
| 23262|    PURA250|           410|
| 23262|    PURA500|           534|
| 23262|    SUPA101|           351|
| 23262|    SUPA105|           239|
| 23262|    SUPA104|           239|
| 23262|    SUPA103|           250|
| 23262|    DETA100|           338|
+------+-----------+--------------+
only showing top 10 rows



In [724]:
w = W.partitionBy('custId').orderBy(f.col("totalUnitsSold").desc())
df3 = df2.withColumn("row", F.row_number().over(w)).filter(F.col("row") == 1).drop("row")

In [725]:
df3 = df3.withColumnRenamed('productSold', 'favourite_product').drop("totalUnitsSold")

In [726]:
df3.show(10)

+------+-----------------+
|custId|favourite_product|
+------+-----------------+
| 23262|          DETA800|
| 23263|          PURA100|
| 23264|          PURA500|
| 23266|          SUPA104|
| 23267|          PURA100|
| 23268|          PURA250|
| 23269|          SUPA101|
| 23270|          PURA500|
| 23271|          PURA250|
| 23273|          PURA100|
+------+-----------------+
only showing top 10 rows



In [727]:
fav_df = df3.alias('fav_df')

# Longest Streak

In [656]:
# data = [
#     ("Alice", date.fromisoformat("2023-01-01")),
#     ("Alice", date.fromisoformat("2023-01-02")),
#     ("Alice", date.fromisoformat("2023-01-03")),
#     ("Alice", date.fromisoformat("2023-01-06")),
#     ("Alice", date.fromisoformat("2023-01-07")),
#     ("Alice", date.fromisoformat("2023-01-08")),
#     ("Foo", date.fromisoformat("2023-01-05")),
#     ("Foo", date.fromisoformat("2023-01-06")),
#     ("Foo", date.fromisoformat("2023-01-07")),
#     ("Foo", date.fromisoformat("2023-01-07")),
#     ("Foo", date.fromisoformat("2023-01-09")),
#     ("Bar", date.fromisoformat("2023-01-09"))
# ]
# df = spark.createDataFrame(data, schema="custId string, transactionDate date")

In [657]:
df.columns

['transactionId', 'custId', 'transactionDate', 'productSold', 'unitsSold']

In [658]:
df.show()

+----------------+------+---------------+-----------+---------+
|   transactionId|custId|transactionDate|productSold|unitsSold|
+----------------+------+---------------+-----------+---------+
|20110111-0023586| 23586|     2011-01-11|    DETA800|      176|
|20110111-0023586| 23586|     2011-01-11|    SUPA101|      193|
|20110111-0023586| 23586|     2011-01-11|    SUPA102|      157|
|20110113-0023524| 23524|     2011-01-13|    PURA250|      122|
|20110113-0023844| 23844|     2011-01-13|    SUPA101|      217|
|20110114-0023968| 23968|     2011-01-14|    PURA100|      284|
|20110119-0023582| 23582|     2011-01-19|    PURA100|      184|
|20110119-0023582| 23582|     2011-01-19|    PURA250|      178|
|20110119-0023582| 23582|     2011-01-19|    SUPA101|      173|
|20110119-0023582| 23582|     2011-01-19|    SUPA101|      173|
|20110119-0023582| 23582|     2011-01-19|    SUPA101|      173|
|20110123-0023572| 23572|     2011-01-23|    PURA500|       90|
|20110124-0023861| 23861|     2011-01-24

In [675]:
df2 = df.groupby(['custId','transactionDate']).agg(F.count('transactionId').alias('trxCount'))

In [676]:
df2.filter(F.col("custId") == '23262').show(1000)

+------+---------------+--------+
|custId|transactionDate|trxCount|
+------+---------------+--------+
| 23262|     2012-09-11|       3|
| 23262|     2012-01-30|       1|
| 23262|     2011-11-06|       2|
| 23262|     2012-05-17|       1|
| 23262|     2011-11-23|       1|
| 23262|     2012-02-15|       2|
| 23262|     2012-01-17|       2|
| 23262|     2011-12-16|       3|
| 23262|     2012-01-22|       2|
| 23262|     2011-11-30|       2|
| 23262|     2012-01-13|       3|
| 23262|     2012-07-02|       2|
| 23262|     2012-07-15|       3|
| 23262|     2012-07-16|       3|
| 23262|     2012-09-30|       2|
+------+---------------+--------+



In [677]:
w = W.partitionBy('custId').orderBy('transactionDate')
df2 = df2.withColumn('transactionDate', F.to_date('transactionDate', format='yyyy-MM-dd'))

In [678]:
df2 = df2.orderBy(["custId", "transactionDate"])

In [679]:
diff = F.datediff('transactionDate', F.lag('transactionDate').over(w))
df3 = df2.withColumn('datediff', F.coalesce(diff))

In [680]:
df3 = df3.withColumn('datediff', F.coalesce(diff, F.lit(0)))

In [682]:
# df = df.withColumn('isConsecutive', F.when(F.col('datediff').isNull(), False).when(F.col('datediff') == 0, True).when(F.col('datediff') == 1, True).otherwise(False))
df3 = df3.withColumn('isConsecutive', F.when(F.col('datediff') == 1, True).otherwise(False))

In [685]:
df3.show(10)

+------+---------------+--------+--------+-------------+
|custId|transactionDate|trxCount|datediff|isConsecutive|
+------+---------------+--------+--------+-------------+
| 23262|     2011-11-06|       2|       0|        false|
| 23262|     2011-11-23|       1|      17|        false|
| 23262|     2011-11-30|       2|       7|        false|
| 23262|     2011-12-16|       3|      16|        false|
| 23262|     2012-01-13|       3|      28|        false|
| 23262|     2012-01-17|       2|       4|        false|
| 23262|     2012-01-22|       2|       5|        false|
| 23262|     2012-01-30|       1|       8|        false|
| 23262|     2012-02-15|       2|      16|        false|
| 23262|     2012-05-17|       1|      92|        false|
+------+---------------+--------+--------+-------------+
only showing top 10 rows



In [688]:
df3 = df3.withColumn('group', F.sum((~F.col('isConsecutive')).cast('int')).over(w))

In [689]:
df3.filter(F.col("custId") == '23262').show(1000)

+------+---------------+--------+--------+-------------+-----+
|custId|transactionDate|trxCount|datediff|isConsecutive|group|
+------+---------------+--------+--------+-------------+-----+
| 23262|     2011-11-06|       2|       0|        false|    1|
| 23262|     2011-11-23|       1|      17|        false|    2|
| 23262|     2011-11-30|       2|       7|        false|    3|
| 23262|     2011-12-16|       3|      16|        false|    4|
| 23262|     2012-01-13|       3|      28|        false|    5|
| 23262|     2012-01-17|       2|       4|        false|    6|
| 23262|     2012-01-22|       2|       5|        false|    7|
| 23262|     2012-01-30|       1|       8|        false|    8|
| 23262|     2012-02-15|       2|      16|        false|    9|
| 23262|     2012-05-17|       1|      92|        false|   10|
| 23262|     2012-07-02|       2|      46|        false|   11|
| 23262|     2012-07-15|       3|      13|        false|   12|
| 23262|     2012-07-16|       3|       1|         true

In [702]:
df4 = df3.groupBy('custID','group').agg(F.sum(F.lit(1)).alias('longestStreak'))

In [704]:
df4.groupBy('custID').agg(F.max('longestStreak')).show()

+------+------------------+
|custID|max(longestStreak)|
+------+------------------+
| 23262|                 2|
| 23263|                 2|
| 23264|                 1|
| 23266|                 1|
| 23267|                 2|
| 23268|                 1|
| 23269|                 1|
| 23270|                 1|
| 23271|                 1|
| 23273|                 1|
| 23274|                 1|
| 23275|                 1|
| 23276|                 1|
| 23277|                 1|
| 23278|                 2|
| 23279|                 1|
| 23280|                 1|
| 23281|                 1|
| 23282|                 1|
| 23283|                 1|
+------+------------------+
only showing top 20 rows



In [735]:
con_df = df4.groupBy('custID').agg(F.max('longestStreak').alias('longestStreak'))

# Aggregation

In [736]:
con_df.show(5)

+------+-------------+
|custID|longestStreak|
+------+-------------+
| 23262|            2|
| 23263|            2|
| 23264|            1|
| 23266|            1|
| 23267|            2|
+------+-------------+
only showing top 5 rows



In [737]:
con_df.count()

885

In [738]:
fav_df.show(5)

+------+-----------------+
|custId|favourite_product|
+------+-----------------+
| 23262|          DETA800|
| 23263|          PURA100|
| 23264|          PURA500|
| 23266|          SUPA104|
| 23267|          PURA100|
+------+-----------------+
only showing top 5 rows



In [734]:
fav_df.count()

885

In [746]:
fav_df.join(con_df, ['custId']).orderBy('custId').show()

+------+-----------------+-------------+
|custId|favourite_product|longestStreak|
+------+-----------------+-------------+
| 23262|          DETA800|            2|
| 23263|          PURA100|            2|
| 23264|          PURA500|            1|
| 23266|          SUPA104|            1|
| 23267|          PURA100|            2|
| 23268|          PURA250|            1|
| 23269|          SUPA101|            1|
| 23270|          PURA500|            1|
| 23271|          PURA250|            1|
| 23273|          PURA100|            1|
| 23274|          PURA100|            1|
| 23275|          PURA500|            1|
| 23276|          PURA500|            1|
| 23277|          PURA100|            1|
| 23278|          SUPA101|            2|
| 23279|          PURA500|            1|
| 23280|          PURA100|            1|
| 23281|          PURA500|            1|
| 23282|          PURA500|            1|
| 23283|          PURA100|            1|
+------+-----------------+-------------+
only showing top

In [747]:
final_df = fav_df.join(con_df, ['custId']).orderBy('custId')

# Test

In [ ]:
# 23262

In [693]:
df.filter(F.col("custId") == '23278').show(1000)

+----------------+------+---------------+-----------+---------+
|   transactionId|custId|transactionDate|productSold|unitsSold|
+----------------+------+---------------+-----------+---------+
|20120205-0023278| 23278|     2012-02-05|    PURA250|      214|
|20120205-0023278| 23278|     2012-02-05|    SUPA101|      191|
|20120205-0023278| 23278|     2012-02-05|    SUPA103|      167|
|20120208-0023278| 23278|     2012-02-08|    DETA800|      196|
|20120208-0023278| 23278|     2012-02-08|    PURA100|      188|
|20120310-0023278| 23278|     2012-03-10|    SUPA101|      254|
|20120324-0023278| 23278|     2012-03-24|    SUPA101|      249|
|20120325-0023278| 23278|     2012-03-25|    PURA100|      228|
|20120325-0023278| 23278|     2012-03-25|    PURA250|      212|
|20120325-0023278| 23278|     2012-03-25|    SUPA102|      199|
|20120517-0023278| 23278|     2012-05-17|    PURA100|      224|
|20120601-0023278| 23278|     2012-06-01|    PURA100|      225|
|20120721-0023278| 23278|     2012-07-21